In [3]:
import os
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from urllib.request import urlretrieve


<h3>Telechargement

In [4]:
url='https://public.opendatasoft.com/explore/dataset/code-insee-postaux-geoflar/download/?format=csv&q=94&refine.nom_region=ILE-DE-FRANCE&refine.nom_dept=VAL-DE-MARNE&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B'
df = pd.read_csv(url,sep=';')  #Si separateur =;
df.head()

,CODE INSEE,NOM DE LA COMMUNE,CODE POSTAL,LIBELLE D'ACHEMINEMENT,Y Centroid,Nom Region,Nom Dept,STATUT,Code Region,Code Commune,...,Geometry,Id Geofla,X Chef Lieu,Code Canton,Superficie,Population,Nom Commune,Code Arrondissement,Y Chef Lieu,Code commune complet
0,94016,CACHAN,94230,CACHAN,68548,ILE-DE-FRANCE,VAL-DE-MARNE,Chef-lieu canton,11,16,...,"{""type"": ""Polygon"", ""coordinates"": [[[2.325812...",16649,6509,5,278,27.8,CACHAN,3,68548,94016
1,94075,VILLECRESNES,94440,VILLECRESNES,68468,ILE-DE-FRANCE,VAL-DE-MARNE,Chef-lieu canton,11,75,...,"{""type"": ""Polygon"", ""coordinates"": [[[2.529578...",21297,6657,26,564,9.5,VILLECRESNES,1,68468,94075
2,94022,CHOISY LE ROI,94600,CHOISY LE ROI,68517,ILE-DE-FRANCE,VAL-DE-MARNE,Chef-lieu canton,11,22,...,"{""type"": ""Polygon"", ""coordinates"": [[[2.427516...",36172,6564,9,541,39.4,CHOISY-LE-ROI,1,68519,94022
3,94070,SANTENY,94440,SANTENY,68484,ILE-DE-FRANCE,VAL-DE-MARNE,Commune simple,11,70,...,"{""type"": ""Polygon"", ""coordinates"": [[[2.568123...",31361,6685,26,1001,3.7,SANTENY,1,68473,94070
4,94033,FONTENAY SOUS BOIS,94120,FONTENAY SOUS BOIS,68613,ILE-DE-FRANCE,VAL-DE-MARNE,Chef-lieu canton,11,33,...,"{""type"": ""Polygon"", ""coordinates"": [[[2.467245...",2273,6616,91,557,53.3,FONTENAY-SOUS-BOIS,2,68614,94033


<h2>Analyse

In [5]:

list(df)

['CODE INSEE',
 'NOM DE LA COMMUNE',
 'CODE POSTAL',
 "LIBELLE D'ACHEMINEMENT",
 'Y Centroid',
 'Nom Region',
 'Nom Dept',
 'STATUT',
 'Code Region',
 'Code Commune',
 'Z Moyen',
 'Code Dept',
 'X Centroid',
 'Geometry X Y',
 'Geometry',
 'Id Geofla',
 'X Chef Lieu',
 'Code Canton',
 'Superficie',
 'Population',
 'Nom Commune',
 'Code Arrondissement',
 'Y Chef Lieu',
 'Code commune complet']

In [6]:
df.shape

(48, 24)

In [7]:
df.dtypes

CODE INSEE                  int64
NOM DE LA COMMUNE          object
CODE POSTAL                 int64
LIBELLE D'ACHEMINEMENT     object
Y Centroid                  int64
Nom Region                 object
Nom Dept                   object
STATUT                     object
Code Region                 int64
Code Commune                int64
Z Moyen                     int64
Code Dept                   int64
X Centroid                  int64
Geometry X Y               object
Geometry                   object
Id Geofla                   int64
X Chef Lieu                 int64
Code Canton                 int64
Superficie                  int64
Population                float64
Nom Commune                object
Code Arrondissement         int64
Y Chef Lieu                 int64
Code commune complet        int64
dtype: object

<h3>Changement d'un type de colonne

In [8]:
df=df.astype({'CODE INSEE': str})  # le titre de colonne peut etre un integer, a ce moment la ne pas mettre de guillemets
df.dtypes

CODE INSEE                 object
NOM DE LA COMMUNE          object
CODE POSTAL                 int64
LIBELLE D'ACHEMINEMENT     object
Y Centroid                  int64
Nom Region                 object
Nom Dept                   object
STATUT                     object
Code Region                 int64
Code Commune                int64
Z Moyen                     int64
Code Dept                   int64
X Centroid                  int64
Geometry X Y               object
Geometry                   object
Id Geofla                   int64
X Chef Lieu                 int64
Code Canton                 int64
Superficie                  int64
Population                float64
Nom Commune                object
Code Arrondissement         int64
Y Chef Lieu                 int64
Code commune complet        int64
dtype: object

<h2>Manipulation

In [9]:
dg=df.loc[:,['CODE INSEE','NOM DE LA COMMUNE', 'Geometry X Y','Geometry','Population','Superficie']]
dg.head()

,CODE INSEE,NOM DE LA COMMUNE,Geometry X Y,Geometry,Population,Superficie
0,94016,CACHAN,"48.7916121646,2.33153313164","{""type"": ""Polygon"", ""coordinates"": [[[2.325812...",27.8,278
1,94075,VILLECRESNES,"48.7208692486,2.53157741439","{""type"": ""Polygon"", ""coordinates"": [[[2.529578...",9.5,564
2,94022,CHOISY LE ROI,"48.764315118,2.41742518222","{""type"": ""Polygon"", ""coordinates"": [[[2.427516...",39.4,541
3,94070,SANTENY,"48.7356177142,2.57637480242","{""type"": ""Polygon"", ""coordinates"": [[[2.568123...",3.7,1001
4,94033,FONTENAY SOUS BOIS,"48.8511046382,2.47395409774","{""type"": ""Polygon"", ""coordinates"": [[[2.467245...",53.3,557


In [10]:
dh=dg['Geometry X Y'].str.split(",", expand=True)
dg=dg.join(dh)
dg.rename(columns={0:'X',1:'Y'},inplace=True)
dg.drop(['Geometry X Y', 'Geometry'], axis=1,inplace=True)
dg.head()


,CODE INSEE,NOM DE LA COMMUNE,Population,Superficie,X,Y
0,94016,CACHAN,27.8,278,48.7916121646,2.33153313164
1,94075,VILLECRESNES,9.5,564,48.7208692486,2.53157741439
2,94022,CHOISY LE ROI,39.4,541,48.764315118,2.41742518222
3,94070,SANTENY,3.7,1001,48.7356177142,2.57637480242
4,94033,FONTENAY SOUS BOIS,53.3,557,48.8511046382,2.47395409774


<h3> set index

In [12]:
dg.set_index('CODE INSEE',inplace=True)
dg.head()

,NOM DE LA COMMUNE,Population,Superficie,X,Y
CODE INSEE,,,,,
94016,CACHAN,27.8,278,48.7916121646,2.33153313164
94075,VILLECRESNES,9.5,564,48.7208692486,2.53157741439
94022,CHOISY LE ROI,39.4,541,48.764315118,2.41742518222
94070,SANTENY,3.7,1001,48.7356177142,2.57637480242
94033,FONTENAY SOUS BOIS,53.3,557,48.8511046382,2.47395409774


<h3> reset index

In [ ]:
df = df.reset_index(drop=True)

<h2>Hot_encoding

In [10]:
dp = pd.DataFrame({'country': ['russia', 'germany', 'australia','korea','germany']})
dp.head()

,country
0,russia
1,germany
2,australia
3,korea
4,germany


In [11]:
dph=pd.get_dummies(dp['country'], prefix='country')
dph.head()


,country_australia,country_germany,country_korea,country_russia
0,0,0,0,1
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,0,1,0,0


<h3>Tri

In [12]:
dg.sort_values(by = 'CODE INSEE', inplace = True,kind = 'mergesort')
dg.head(5)

,CODE INSEE,NOM DE LA COMMUNE,Population,Superficie,X,Y
15,94001,ABLON SUR SEINE,5.2,114,48.7238409464,2.42117629348
26,94002,ALFORTVILLE,44.3,368,48.7960843864,2.42124593241
11,94003,ARCUEIL,19.5,232,48.8058803597,2.33351024984
28,94004,BOISSY ST LEGER,16.2,888,48.7470359922,2.52528049019
32,94011,BONNEUIL SUR MARNE,16.6,553,48.7733620104,2.48799310463


<h2>Dropna

In [13]:
#dv=du.dropna()      pour l'ensemble 
dv=du.dropna(subset=['Terrain'])
dv.head()

NameError: name 'du' is not defined

<h2>Calcul

In [14]:
dg['Densite']=dg['Population']/dg['Superficie']*1000
dg.head()

,CODE INSEE,NOM DE LA COMMUNE,Population,Superficie,X,Y,Densite
15,94001,ABLON SUR SEINE,5.2,114,48.7238409464,2.42117629348,45.614035
26,94002,ALFORTVILLE,44.3,368,48.7960843864,2.42124593241,120.380435
11,94003,ARCUEIL,19.5,232,48.8058803597,2.33351024984,84.051724
28,94004,BOISSY ST LEGER,16.2,888,48.7470359922,2.52528049019,18.243243
32,94011,BONNEUIL SUR MARNE,16.6,553,48.7733620104,2.48799310463,30.018083


<h3>Extraction de lignes conditionelle

In [15]:
dg=dg[dg['Population']>70]
#df2 = df[(df.a != -1) | (df.b != -1)] condition multiple
dg.head()

,CODE INSEE,NOM DE LA COMMUNE,Population,Superficie,X,Y,Densite
31,94017,CHAMPIGNY SUR MARNE,75.1,1130,48.817254364,2.51709815257,66.460177
19,94028,CRETEIL,89.4,1144,48.7837401836,2.45463530415,78.146853
17,94068,ST MAUR DES FOSSES,75.3,1123,48.7990677797,2.49386450452,67.052538
20,94068,ST MAUR DES FOSSES,75.3,1123,48.7990677797,2.49386450452,67.052538
14,94081,VITRY SUR SEINE,85.4,1165,48.7882828307,2.39412680533,73.304721


<h2>Reorganisation des colonnes ou extraction de colonnes

In [65]:
cols = dg.columns.tolist()
cols

['CODE INSEE',
 'NOM DE LA COMMUNE',
 'Population',
 'Superficie',
 'X',
 'Y',
 'Densite']

In [67]:
cols=['CODE INSEE', 'NOM DE LA COMMUNE','Population', 'Densite', 'X', 'Y']
dg = dg[cols]
dg.head()

,CODE INSEE,NOM DE LA COMMUNE,Population,Densite,X,Y
0,94028,CRETEIL,89.4,78.146853,48.7837401836,2.45463530415
1,94068,ST MAUR DES FOSSES,75.3,67.052538,48.7990677797,2.49386450452
2,94044,LIMEIL BREVANNES,19.1,27.403156,48.7450357782,2.48936184357
3,94073,THIAIS,29.5,45.807453,48.7607841928,2.38537729524
4,94002,ALFORTVILLE,44.3,120.380435,48.7960843864,2.42124593241


In [54]:
cols=['CODE INSEE', 'NOM DE LA COMMUNE','Population']
dg = dg[cols]
dg.head()

,CODE INSEE,NOM DE LA COMMUNE,Population
0,94028,CRETEIL,89.4
1,94068,ST MAUR DES FOSSES,75.3
2,94044,LIMEIL BREVANNES,19.1
3,94073,THIAIS,29.5
4,94002,ALFORTVILLE,44.3


<h3>Transformation du type colonne

In [15]:
dh=dg.astype({'Population': int})
#df.astype({'A': int, 'B': numpy.float64})
dh.head()

,NOM DE LA COMMUNE,Population,Superficie,X,Y
CODE INSEE,,,,,
94016,CACHAN,27,278,48.7916121646,2.33153313164
94075,VILLECRESNES,9,564,48.7208692486,2.53157741439
94022,CHOISY LE ROI,39,541,48.764315118,2.41742518222
94070,SANTENY,3,1001,48.7356177142,2.57637480242
94033,FONTENAY SOUS BOIS,53,557,48.8511046382,2.47395409774


<h2>Extraction a partir d'une table html

In [22]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r= requests.get(url)
soup = BeautifulSoup(r.content)
tbl = soup.find_all('table')[0]
df = pd.read_html(str(tbl))[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h2>Parcourir un dataset

In [ ]:
for i,row in df3.iterrows():
    folium.CircleMarker(location=[row["Latitude"],row["Longitude"]],radius=5,color='black').add_to(m)
 
m


<h2>Autre methode

In [ ]:
for latitude, longitude, Postcode in zip(df3['Latitude'], df3['Longitude'], df3['Postcode']):
    label = '{}'.format(Postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m)  
    
m